<a href="https://colab.research.google.com/github/cooolbabu/GoogleGemini101/blob/main/OpenAI/SimpleChatWithPromptTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Chat Models


In [1]:
%pip install langchain langchain_openai --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requi

# All Imports

In [67]:
# All Imports in the single location and API key from colab notebook

from google.colab import userdata
OPENAI_API_KEY=userdata.get('OpenAI_KEY')

In [68]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
)

import pandas as pd
import json

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator

from typing import List

from IPython.display import display, Markdown, Latex

# Let's have a simple chat

In [69]:
chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

chat.invoke("Hello, how are you?")

AIMessage(content="Hello! I'm an AI, so I don't have feelings, but I'm here to help you with any questions or tasks you have. How can I assist you today?")

In [70]:
from langchain_core.messages import HumanMessage, SystemMessage

text = "What would be a good company name for a company that makes colorful socks?"
messages = [HumanMessage(content=text)]
result = chat.invoke(messages)

In [71]:
print(type(result))
print(result)
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
content='VibrantSox'
VibrantSox


---

## Inserting the SystemMessage instead of a HumanMessage


In [72]:
text = "What would be a good company name for a company that makes colorful socks?"

messages = [SystemMessage(content=text)]
chatResponse = chat(messages)
print(chatResponse)

content='Vibrant Feet'


In [73]:
print(type(chatResponse))
print(chatResponse)
print(chatResponse.content)

<class 'langchain_core.messages.ai.AIMessage'>
content='Vibrant Feet'
Vibrant Feet


# __Brand names__ -  Let have a chatbot that uses prompt template to suggest brand names
- We want brand names for our shoes. Different countries have their localized brand name
- Pass country name as a parameter and generate local brand
- Specify that output format is a JSON with root element as brand_names

In [74]:

# Country name is a parameter in the prompt template.

chat = ChatOpenAI(openai_api_key=OPENAI_API_KEY)
prompt_template = "You are shoe marketing specialist in {country}. What would be catch brand name for shoes. Add country native flavor. Suggest three names and return result in json format with name and reason. Use brand_names as root element"


system_message_prompt = SystemMessagePromptTemplate.from_template(prompt_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])



In [75]:
# Generate suggestions for Italian market. 'brand_name' is specified as the root element
italian_suggestions = chat.invoke(
    chat_prompt.format_prompt(country="Italy").to_messages()
)

df = pd.DataFrame(json.loads(italian_suggestions.content)['brand_names'])
df

,name,reason
0,Ferraglio,This brand name combines the elegance and soph...
1,SoleItalia,SoleItalia captures the essence of Italian sho...
2,PiedeForte,PiedeForte translates to 'strong foot' in Ital...


In [76]:
# Generate suggestions for Mexican market
mexican_suggestions = chat.invoke(
    chat_prompt.format_prompt(country="Mexico").to_messages()
)

df = pd.DataFrame(json.loads(mexican_suggestions.content)['brand_names'])
df

,name,reason
0,Piedra Sagrada,This name combines the idea of sacredness (sag...
1,Caminos del Sol,"This name translates to 'Paths of the Sun,' ev..."
2,Alas del Viento,"This name means 'Wings of the Wind,' symbolizi..."


# __Brand Names__ - Sample brand names example using Parsers
- In the previous example we specified the output format to be a JSON object ith brand_names as the root element
- In this section let's see how we can use Langchain parsers
- There are a variety of Output parsers
  - https://python.langchain.com/docs/modules/model_io/output_parsers/
  - Here let's take a look at Pydantic, JSON and Pandas dataframe

- Very useful when calling functions and data needs to be in a structured format

- I haven't seen this as a named pattern. But it looks like Setup and Get responses. Which is same as SystemMessage and User message in OpenAI Playground


In [77]:
class BrandName(BaseModel):
    brand_name: str = Field(description="Suggested a brand name and give reason")
    reason: str = Field(description="Reason for the brand name")


class BrandNames(BaseModel):
    brand_names: List[BrandName] = Field(description="List of Brand names")



In [60]:
parser = PydanticOutputParser(pydantic_object=BrandNames)

In [78]:
template = "Answer the user query.\n{format_instructions}\n{query}\n"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

In [79]:
display(Markdown(parser.get_format_instructions()))

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"BrandName": {"properties": {"brand_name": {"description": "Suggested a brand name and give reason", "title": "Brand Name", "type": "string"}, "reason": {"description": "Reason for the brand name", "title": "Reason", "type": "string"}}, "required": ["brand_name", "reason"], "title": "BrandName", "type": "object"}}, "properties": {"brand_names": {"description": "List of Brand names", "items": {"$ref": "#/$defs/BrandName"}, "title": "Brand Names", "type": "array"}}, "required": ["brand_names"]}
```

In [80]:
# Format the chat prompt:
messages = chat_prompt.format_prompt(
    format_instructions=parser.get_format_instructions(),
    query="You are a Maketing specialist in Mexico. Come up with 3 brand names for a shoe. Use catchy and local colloquial language",
).to_messages()

In [81]:
# What are we send to LLM
print(messages)

[SystemMessage(content='Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"BrandName": {"properties": {"brand_name": {"description": "Suggested a brand name and give reason", "title": "Brand Name", "type": "string"}, "reason": {"description": "Reason for the brand name", "title": "Reason", "type": "string"}}, "required": ["brand_name", "reason"], "title": "BrandName", "type": "object"}}, "properties": {"brand_names": {"description": "List of Brand names", "items": {"$ref": "#/$defs/BrandName"}, "title": "Brand Names", "type": "array"}}, "required": ["

In [65]:
chat_response = chat.invoke(messages)

In [82]:
# What is the response from Chat model?

print(chat_response.content)

{
  "brand_names": [
    {
      "brand_name": "Pata Chula",
      "reason": "This brand name translates to 'Cool Paw' in English, and it has a catchy and playful tone that appeals to Mexican consumers. It also relates to the product being shoes."
    },
    {
      "brand_name": "Zapato Loco",
      "reason": "Meaning 'Crazy Shoe' in English, this brand name has a fun and energetic vibe. It captures attention and stands out in the market, appealing to the younger generation."
    },
    {
      "brand_name": "Soleado",
      "reason": "This brand name means 'Sunny' in English. It conveys a sense of warmth, comfort, and positivity associated with wearing shoes. It resonates with the Mexican culture's love for sunshine and outdoor activities."
    }
  ]
}


## Another example of using setup and get responses

In [83]:

class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


class Jokes(BaseModel):
    jokes: List[Joke] = Field(description="list of jokes")

In [84]:
parser = PydanticOutputParser(pydantic_object=Jokes)

In [85]:
template = "Answer the user query.\n{format_instructions}\n{query}\n"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

In [86]:
# Pydantic parser creates an json schema file, which then can be passed as instructions to LLMs
# Output is a Markdown format. Markdown lib is used for better readability

display(Markdown(parser.get_format_instructions()))

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Joke": {"properties": {"setup": {"description": "question to set up a joke", "title": "Setup", "type": "string"}, "punchline": {"description": "answer to resolve the joke", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"], "title": "Joke", "type": "object"}}, "properties": {"jokes": {"description": "list of jokes", "items": {"$ref": "#/$defs/Joke"}, "title": "Jokes", "type": "array"}}, "required": ["jokes"]}
```

In [87]:
# Format the chat prompt:
messages = chat_prompt.format_prompt(
    format_instructions=parser.get_format_instructions(),
    query="What's really funny about Python programming? Give 3 jokes",
).to_messages()

In [88]:
result = chat.invoke(messages)

In [89]:
print(messages)

[SystemMessage(content='Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Joke": {"properties": {"setup": {"description": "question to set up a joke", "title": "Setup", "type": "string"}, "punchline": {"description": "answer to resolve the joke", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"], "title": "Joke", "type": "object"}}, "properties": {"jokes": {"description": "list of jokes", "items": {"$ref": "#/$defs/Joke"}, "title": "Jokes", "type": "array"}}, "required": ["jokes"]}\n```\nWhat\'s really funny about Python pr

In [90]:
print(result.content)

{"jokes": [
    {
        "setup": "Why do Python programmers prefer using snake_case?",
        "punchline": "Because they don't like Java."
    },
    {
        "setup": "Why was the Python programmer not happy with his job?",
        "punchline": "He wanted more inheritance."
    },
    {
        "setup": "What do you call a snake that is a programmer?",
        "punchline": "A Python."
    }
]}


# What is the difference between OpenAI and ChatOpenAI

- OpenAI class includes more generic machine learning task attributes such as frequency_penalty, presence_penalty, logit_bias, allowed_special, disallowed_special, best_of.

- ChatOpenAI class provides more chat-related methods, such as completion_with_retry, get_num_tokens_from_messages to make it more user-friendly when build chatbot related applications.

[Good article in Stack Overflow](https://stackoverflow.com/questions/76950609/what-is-the-difference-between-openai-and-chatopenai-in-langchain)



# Scratchpad

In [24]:
from IPython.display import display, Markdown, Latex
display(Markdown('*Italics Text* __Bold Text__'))
# If you particularly want to display maths, this is more direct:
display(Latex('\phi'))
display(Markdown(parser.get_format_instructions()))

*Italics Text* __Bold Text__

<IPython.core.display.Latex object>

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Joke": {"properties": {"setup": {"description": "question to set up a joke", "title": "Setup", "type": "string"}, "punchline": {"description": "answer to resolve the joke", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"], "title": "Joke", "type": "object"}}, "properties": {"jokes": {"description": "list of jokes", "items": {"$ref": "#/$defs/Joke"}, "title": "Jokes", "type": "array"}}, "required": ["jokes"]}
```